#1.도착여부 예측

* 고객 구매 데이터를 사용해서 고객이 주문한 물품이 제시간에 도착역부(Reache.on.Time_Y.N) 예측
* X_train.csv,y_train_csv : 학습 및 학습평가에 사용하는 파일
* X_test.csv : 제출용 데이터 생성에 사용하는 파일
* roc_score을 사용하여 평
* 아래의 형식으로 csv로 저장
* Reached.on.Time.Y.N은 제시간에 도착할 확률

* ID,Reached.on.Time_Y.N
* 7007,0.2741
* 9793,0.4883
* 6593,0.4817
* 10527,0.3657

In [ ]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/HyeonSu-Kang/data/refs/heads/main/1st_Train.csv')
df

,ID,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms,Reached.on.Time_Y.N
0,1,D,Flight,4,2,177,3,low,F,44,1233,1
1,2,F,Flight,4,5,216,2,low,M,59,3088,1
2,3,A,Flight,2,2,183,4,low,M,48,3374,1
3,4,B,Flight,3,3,176,4,medium,M,10,1177,1
4,5,C,Flight,2,2,184,3,medium,F,46,2484,1
...,...,...,...,...,...,...,...,...,...,...,...,...
10994,10995,A,Ship,4,1,252,5,medium,F,1,1538,1
10995,10996,B,Ship,4,1,232,5,medium,F,6,1247,0
10996,10997,C,Ship,5,4,242,5,low,F,4,1155,0
10997,10998,F,Ship,5,2,223,6,medium,M,2,1210,0


In [ ]:
#[0] x_train,x_test 데이터로 생성함
from sklearn.model_selection import train_test_split
import pandas as pd
dftot = df
x_train,x_test = train_test_split(dftot,test_size=0.4,
                                  stratify = dftot['Reached.on.Time_Y.N'],
                                  random_state=0)
y_train = x_train['Reached.on.Time_Y.N']
x_train = x_train.drop(columns = 'Reached.on.Time_Y.N')
y_test = x_test[['Reached.on.Time_Y.N']]
x_test = x_test.drop(columns = 'Reached.on.Time_Y.N')
x_train.to_csv('x_train.csv',index=False)
x_test.to_csv('x_test.csv',index=False)
y_train.to_csv('y_train.csv',index=False)
y_test.to_csv('y_test.csv',index=False)

In [ ]:
# 사용 라이브러리 import

import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

#데이터가 많은 경우 모두 출력 완되고 ...으로 생략해서 출력됨
pd.set_option('display.max_rows',500)    #출력할 max row를 지정
pd.set_option('display.max_columns',20)  #출력할 max columns를 지정
# 출력 format 지정 - 소수점아래 4자리까지
pd.set_option('display.float_format','{:.4f}'.format)

In [ ]:
#[1] x_train.csv,y_train.csv,x_test 읽어오기
X = pd.read_csv('x_train.csv')
Y = pd.read_csv('y_train.csv')
X_submission = pd.read_csv('x_test.csv')

In [ ]:
#[2] X,X_submission 합치기(필수는 아님)
dfX = pd.concat([X,X_submission],axis=0,ignore_index=True)
display(dfX.head(3))

#[3] df의 정보 확인
print(dfX.info())

#[4] 결측치 점검
print(dfX.isna().sum())

#[5] X,Y를 합쳐 dfXY를 만듬
dfXY = pd.concat([dfX,Y])
print(dfXY.shape)

,ID,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms
0,9902,F,Ship,3,5,214,2,medium,F,6,4578
1,9501,A,Ship,4,2,201,3,medium,F,4,4613
2,6111,A,Ship,4,2,264,3,low,M,10,5137


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10999 entries, 0 to 10998
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ID                   10999 non-null  int64 
 1   Warehouse_block      10999 non-null  object
 2   Mode_of_Shipment     10999 non-null  object
 3   Customer_care_calls  10999 non-null  int64 
 4   Customer_rating      10999 non-null  int64 
 5   Cost_of_the_Product  10999 non-null  int64 
 6   Prior_purchases      10999 non-null  int64 
 7   Product_importance   10999 non-null  object
 8   Gender               10999 non-null  object
 9   Discount_offered     10999 non-null  int64 
 10  Weight_in_gms        10999 non-null  int64 
dtypes: int64(7), object(4)
memory usage: 945.4+ KB
None
ID                     0
Warehouse_block        0
Mode_of_Shipment       0
Customer_care_calls    0
Customer_rating        0
Cost_of_the_Product    0
Prior_purchases        0
Product_importance    

In [ ]:
#[6] dfX의 각 컬럼의 값 종류 개수 - 1인것 제거하기 위해
print(dfX.nunique())

ID                     10999
Warehouse_block            5
Mode_of_Shipment           3
Customer_care_calls        6
Customer_rating            5
Cost_of_the_Product      215
Prior_purchases            8
Product_importance         3
Gender                     2
Discount_offered          65
Weight_in_gms           4034
dtype: int64


In [ ]:
#[7] Object 타입에 대한 Label Encoding
df_LE = dfX.copy()

print(dfX['Warehouse_block'].unique())
df_LE['Warehouse_block'] = dfX['Warehouse_block'].replace(['A','B','C','D','F'],[0,1,2,3,4]).astype('int64')

print(dfX['Mode_of_Shipment'].unique())
df_LE['Mode_of_Shipment'] = dfX['Mode_of_Shipment'].replace(['Ship','Road','Flight'],[0,1,2]).astype('int64')

print(dfX['Product_importance'].unique())
df_LE['Product_importance'] = dfX['Product_importance'].replace(['low','medium','high'],[0,1,2]).astype('int64')

print(dfX['Gender'].unique())
df_LE['Gender'] = dfX['Gender'].replace(['F','M'],[0,1]).astype('int64')

df_LE.info()

['F' 'A' 'B' 'D' 'C']
['Ship' 'Road' 'Flight']
['medium' 'low' 'high']
['F' 'M']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10999 entries, 0 to 10998
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype
---  ------               --------------  -----
 0   ID                   10999 non-null  int64
 1   Warehouse_block      10999 non-null  int64
 2   Mode_of_Shipment     10999 non-null  int64
 3   Customer_care_calls  10999 non-null  int64
 4   Customer_rating      10999 non-null  int64
 5   Cost_of_the_Product  10999 non-null  int64
 6   Prior_purchases      10999 non-null  int64
 7   Product_importance   10999 non-null  int64
 8   Gender               10999 non-null  int64
 9   Discount_offered     10999 non-null  int64
 10  Weight_in_gms        10999 non-null  int64
dtypes: int64(11)
memory usage: 945.4 KB


<ipython-input-7-565f8f1af378>:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_LE['Warehouse_block'] = dfX['Warehouse_block'].replace(['A','B','C','D','F'],[0,1,2,3,4]).astype('int64')
<ipython-input-7-565f8f1af378>:8: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_LE['Mode_of_Shipment'] = dfX['Mode_of_Shipment'].replace(['Ship','Road','Flight'],[0,1,2]).astype('int64')
<ipython-input-7-565f8f1af378>:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old 

In [ ]:
#[8] train score,test score,roc_auc_score 를 반환하는 함수 작성
def get_scores(model,xtrain,xtest,ytrain,ytest):
  A = model.score(xtrain,ytrain)
  B = model.score(xtest,ytest)
  ypred = model.predict_proba(xtest)[:,1]
  C = roc_auc_score(ytest,ypred)
  return '{:.4f} {:.4f} {:.4f}'.format(A,B,C)

In [ ]:
#[9] 다양한 모델을 만들고 성능을 출력하는 함수 작성
def make_models(xtrain,xtest,ytrain,ytest):
  model_1 = LogisticRegression().fit(xtrain,ytrain)
  print('model_1',get_scores(model_1,xtrain,xtest,ytrain,ytest))

  for k in range(1,10):
    model_2 = KNeighborsClassifier(k).fit(xtrain,ytrain)
    print('model_2',get_scores(model_2,xtrain,xtest,ytrain,ytest))

  #overfitting
  model_3 = DecisionTreeClassifier(random_state=0).fit(xtrain,ytrain)
  print('model_3',get_scores(model_3,xtrain,xtest,ytrain,ytest))

  #overfitting 해결
  for k in range(3,8):
    model_3 = DecisionTreeClassifier(max_depth=k,random_state=0).fit(xtrain,ytrain)
    print('model_3',get_scores(model_3,xtrain,xtest,ytrain,ytest))

  #overfitting
  model_4 =  RandomForestClassifier(random_state=0).fit(xtrain,ytrain)
  print('model_4',get_scores(model_4,xtrain,xtest,ytrain,ytest))

  #overfitting 해결
  for d in range(3,8):
    model_4 = RandomForestClassifier(500,random_state=0,max_depth=d).fit(xtrain,ytrain)
    print('model_4',get_scores(model_4,xtrain,xtest,ytrain,ytest))

  model_5 = XGBClassifier().fit(xtrain,ytrain)
  print('model_5',get_scores(model_5,xtrain,xtest,ytrain,ytest))



In [ ]:
#[10] X를 train용도, submission 용도로 나누고, Y를 1차원으로 바꿈(MinMaxScaler)적용
def get_data(dfX,Y):
  X = dfX.drop(columns=['ID'])
  X_use = X.iloc[:6599,:]
  X_submission = X.iloc[6599:,:]
  Y1 = Y['Reached.on.Time_Y.N']
  scaler = MinMaxScaler()
  X1_use = scaler.fit_transform(X_use)
  X1_submission = scaler.transform(X_submission)
  print(X1_use.shape,Y1.shape,X1_submission.shape)
  return X1_use,X1_submission,Y1


In [ ]:
#[11] df_LE 사용
#X,Y 분리하기
X1_use,X1_submission,Y1 = get_data(df_LE,Y)

# train,test 7:3 분할, stratify적용, random_state=0 적용
xtrain1,xtest1,ytrain1,ytest1 = train_test_split(X1_use,Y1,test_size=0.3,stratify=Y1,random_state=0)

# 다양한 모델 만들어 보기 = 성능 확인 후 꼭 주석
k = make_models(xtrain1,xtest1,ytrain1,ytest1)

(6599, 10) (6599,) (4400, 10)
model_1 0.6506 0.6419 0.7224
model_2 1.0000 0.6288 0.6109
model_2 0.8088 0.6379 0.6674
model_2 0.8097 0.6434 0.6922
model_2 0.7647 0.6389 0.6993
model_2 0.7707 0.6439 0.7042
model_2 0.7465 0.6495 0.7124
model_2 0.7463 0.6485 0.7183
model_2 0.7341 0.6515 0.7185
model_2 0.7296 0.6414 0.7149
model_3 1.0000 0.6364 0.6199
model_3 0.6776 0.6763 0.7429
model_3 0.6820 0.6783 0.7437
model_3 0.6893 0.6874 0.7487
model_3 0.6939 0.6747 0.7404
model_3 0.7077 0.6717 0.7359
model_4 1.0000 0.6551 0.7285
model_4 0.6779 0.6732 0.7464
model_4 0.6954 0.6758 0.7438
model_4 0.7058 0.6773 0.7437
model_4 0.7212 0.6773 0.7432
model_4 0.7456 0.6773 0.7414
model_5 0.9504 0.6515 0.7388


In [ ]:
#model_4 0.6893 0.6874 0.7487
#[12] 최종적으로 사용할 모델을 선택하여 model로 다시 생성
model = DecisionTreeClassifier(max_depth=5).fit(xtrain1,ytrain1)
print('final_model',get_scores(model,xtrain1,xtest1,ytrain1,ytest1))

final_model 0.6893 0.6874 0.7487


In [ ]:
#[13] test 데이터에 대한 확률 구하기, 파일로 저장하기
ypred = model.predict_proba(X1_submission)[:,1]
submission = pd.DataFrame({
    "ID" : X_submission["ID"],
    "Reached.on.Time_Y.N" : ypred
})

submission.to_csv('submission_number.csv',index=False)

In [ ]:
#[14] submission확인
temp = pd.read_csv('submission_number.csv')
print(submission.head())

      ID  Reached.on.Time_Y.N
0   7007               0.4627
1   9793               0.4627
2   6593               0.4656
3  10527               0.3921
4   6914               0.4656
